In [ ]:
import torch
import numpy as np
from torch.functional import F
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchvision import transforms
import torchvision.datasets as dst
from torchvision.utils import save_image
import pandas as pd

class vae_net(nn.Module):
    def __init__(self,code_feats):
        super(vae_net,self).__init__()
        self.encode_layer1=nn.Conv2d(in_channels=1,out_channels=32,kernel_size=4,stride=2,padding=1)
        self.encode_layer2=nn.Conv2d(in_channels=32,out_channels=64,kernel_size=4,stride=2,padding=1)
        self.encode_layer3=nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,stride=1,padding=1)
        self.relu=nn.LeakyReLU(0.2, inplace=True)
        self.fc_mean=nn.Linear(in_features=64*7*7,out_features=code_feats)
        self.fc_logvar=nn.Linear(in_features=64*7*7,out_features=code_feats)
        
        #self.batchnorm1=nn.BatchNorm2d(32)
        #self.batchnorm2=nn.BatchNorm2d(64)
        #self.batchnorm3=nn.BatchNorm2d(64)
        
        self.decode_layer1=nn.Linear(code_feats,64*7*7)
        self.decode_layer2=nn.ConvTranspose2d(64, 64, kernel_size=4, stride=2, padding=1)
        self.decode_layer3=nn.ConvTranspose2d(64, 1, kernel_size=4, stride=2, padding=1)
                
    def encode(self,x):
        batch_n=x.shape[0]
        temp=self.encode_layer1(x)
        #temp=self.batchnorm1(temp)
        temp=self.relu(temp)
        temp=self.encode_layer2(temp)
        #temp=self.batchnorm2(temp)
        temp=self.relu(temp)
        temp=self.encode_layer3(temp)
        #temp=self.batchnorm3(temp)
        temp=temp.view(batch_n,-1)
        temp=self.relu(temp)
        
        temp_mean=self.fc_mean(temp)
        #temp_mean=F.relu(temp_mean)
 
        temp_logvar=self.fc_logvar(temp)
        #temp_logvar=F.relu(temp_logvar)

        
        return(temp_mean,temp_logvar)
    
    def decode(self,x):
        temp=self.decode_layer1(x)
        temp=temp.view(-1,64,7,7)
        temp=F.relu(temp)
        temp=self.decode_layer2(temp)
        temp=F.relu(temp)
        temp=self.decode_layer3(temp)
        temp=F.sigmoid(temp)
        return(temp)
    
    def forward(self,x):
        mean,logvar=self.encode(x)
        if torch.cuda.is_available():
            z=mean + Variable(torch.randn(mean.size(0),mean.size(1)).cuda())*(logvar/2).exp()
        else:
            z=mean + Variable(torch.randn(mean.size(0),mean.size(1)))*(logvar/2).exp()
        rec_x=self.decode(z)
        return(rec_x,mean,logvar)


def Loss(x,rec_x,mean,logvar):
    bce_loss=F.binary_cross_entropy(input=rec_x,target=x,size_average=False)
    bld_loss=0.5*torch.sum(mean.pow(2)+logvar.exp()-logvar-1)
    return(bce_loss+bld_loss)



log_interval=100
EPOCH=50
code_feats=64
BATCH_SIZE=64

if torch.cuda.is_available():
    vae=vae_net(code_feats=code_feats).cuda()
else:
    vae=vae_net(code_feats=code_feats)
    
optimizer=optim.Adam(vae.parameters(),lr=1e-3,betas=(0.9, 0.999), eps=1e-08, weight_decay=0)


def train(EPOCH,model):
    time1=pd.datetime.now()
    total_loss = 0
    for i, (data, _) in enumerate(train_loader, 0):
        if torch.cuda.is_available():
            data = Variable(data).cuda()
        else:
            data = Variable(data)
        optimizer.zero_grad()
        rec_x,mean,logvar=vae.forward(data)
        loss=Loss(data,rec_x,mean,logvar)
        loss.backward()
        optimizer.step()
        total_loss+=loss.detach()
        
        
        
        if i % log_interval == 0:
            if torch.cuda.is_available():
                sample = Variable(torch.randn(64, code_feats)).cuda()
            else:
                sample = Variable(torch.randn(64, code_feats))
                
            sample = vae.decode(sample).cpu()
            save_image(sample.data.view(64, 1, 28, 28),
            'result/sample_' + str(epoch)+'_'+str(i) + '.png')
            print('Train Epoch:{} -- [{}/{} ({:.0f}%)] -- Loss:{:.6f}'.format(
                 epoch, i*len(data), len(train_loader.dataset),
                 100.*i/len(train_loader), loss/len(data)))
 
    print('timespent:',pd.datetime.now()-time1,'====> Epoch: {} Average loss: {:.4f}'.format(epoch, total_loss / len(train_loader.dataset)))
    

transform=transforms.Compose([transforms.ToTensor()])
data_train = dst.MNIST('MNIST_data/', train=True, transform=transform, download=True)
data_test = dst.MNIST('MNIST_data/', train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset=data_train, num_workers=2,batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=data_test, num_workers=2,batch_size=BATCH_SIZE, shuffle=True)

for epoch in range(1, EPOCH):
    train(epoch,vae)        


        

/Users/lekang/anaconda/envs/py3/lib/python3.6/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/Users/lekang/anaconda/envs/py3/lib/python3.6/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Train Epoch:1 -- [0/60000 (0%)] -- Loss:548.059998
Train Epoch:1 -- [6400/60000 (11%)] -- Loss:201.648560
Train Epoch:1 -- [12800/60000 (21%)] -- Loss:171.239288
Train Epoch:1 -- [19200/60000 (32%)] -- Loss:146.694717
Train Epoch:1 -- [25600/60000 (43%)] -- Loss:135.922073
Train Epoch:1 -- [32000/60000 (53%)] -- Loss:127.623108
Train Epoch:1 -- [38400/60000 (64%)] -- Loss:128.317062
Train Epoch:1 -- [44800/60000 (75%)] -- Loss:121.796188
Train Epoch:1 -- [51200/60000 (85%)] -- Loss:120.884048


In [24]:
a=torch.Tensor([1,2,3])
a=Variable(a)

In [22]:
a*torch.exp(a)

tensor([ 2.7183, 14.7781, 60.2566])

In [23]:
a*a.exp()

tensor([ 2.7183, 14.7781, 60.2566])

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn